###Una RNN LSTM a nivel de caracter
Vamos a crear una LSTM a nivel de caracter, este modelo será entrenado caracter a caracter y generará un nuevo texto.

Esta red esta basada en Andrej Karpathy's RNNs e implementada en Torch.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

####Cargamos nuestra data
Vamos a cargar nuestro archivo de texto y convertirlo en enteros para que nuestra red LSMT la use.

In [4]:
#Abrimos la data y leemos el archivo
with open('archivo.txt','r') as f:
  text = f.read()
len(text)

691699

Veamos lor primeros 250 caracteres del texto

In [5]:
text[:250]

'Como ya se ha indicado, las veinte secuencias narrativas siguen por lo común el orden cronológico,\npero una importante excepción tiene lugar precisamente en el relato de los acontecimientos que\nllevaron a la fundación de Macondo, que se encuentra en '

###Tokenización
Vamos a crear dos diccionarios para convertir los caracteres en enteros, al hacer esto será más fácil usarlo para la entrada de nuestra red neuronal.

In [6]:
#Codificamos nuestro texto y mapeamos cada caracter a un entero y viceversa

#Vamos a crear dos diccionarios:
# 1. int_a_char, mapea enteros a caracteres
# 2. char_a_int, mapea caracteres a un único entero

caracteres = tuple(set(text))
int_a_char = dict(enumerate(caracteres))
char_a_int = {ch: ii for ii, ch in int_a_char.items()}

#codificamos el texto
encoded = np.array([char_a_int[ch] for ch in text])

Ahora vemos el mismo texto de ejercicio codificado.

In [7]:
encoded[:250]

array([50,  4, 58,  4, 30, 48, 40, 30, 15, 42, 30, 39, 40, 30, 34, 56, 25,
       34, 24, 40, 25,  4, 17, 30, 11, 40, 15, 30, 31, 42, 34, 56, 12, 42,
       30, 15, 42, 24, 47, 42, 56, 24, 34, 40, 15, 30, 56, 40, 28, 28, 40,
       12, 34, 31, 40, 15, 30, 15, 34, 22, 47, 42, 56, 30, 53,  4, 28, 30,
       11,  4, 30, 24,  4, 58, 57, 56, 30, 42, 11, 30,  4, 28, 25, 42, 56,
       30, 24, 28,  4, 56,  4, 11, 37, 22, 34, 24,  4, 17, 13, 53, 42, 28,
        4, 30, 47, 56, 40, 30, 34, 58, 53,  4, 28, 12, 40, 56, 12, 42, 30,
       42, 43, 24, 42, 53, 24, 34, 37, 56, 30, 12, 34, 42, 56, 42, 30, 11,
       47, 22, 40, 28, 30, 53, 28, 42, 24, 34, 15, 40, 58, 42, 56, 12, 42,
       30, 42, 56, 30, 42, 11, 30, 28, 42, 11, 40, 12,  4, 30, 25, 42, 30,
       11,  4, 15, 30, 40, 24,  4, 56, 12, 42, 24, 34, 58, 34, 42, 56, 12,
        4, 15, 30,  1, 47, 42, 13, 11, 11, 42, 31, 40, 28,  4, 56, 30, 40,
       30, 11, 40, 30, 54, 47, 56, 25, 40, 24, 34, 37, 56, 30, 25, 42, 30,
        2, 40, 24,  4, 56

###Procesar la data
Las redes neuronales solo aceptan números, entonces vamos a tener que hacer que nuestros datos sean números y podemos hacer eso con **one-hot encoded** que significa que cada carácter es convertido a un entero y luego en una columna de vectores que para el entero que correponde tendría un 1 y el resto lleno de 0.

In [8]:
def one_hot_encode(arr,n_labels):
  #Iniciamos el arreglo de codificación
  one_hot = np.zeros((arr.size,n_labels),dtype=np.float32)

  #LLenamos apropiadamente con uno
  one_hot[np.arange(one_hot.shape[0]),arr.flatten()]=1.

  #Finalmente hacemos un reshape
  one_hot = one_hot.reshape((*arr.shape,n_labels))

  return one_hot

In [9]:
#Verificamos que la función es correcta
test_seq = np.array([[3,5,1]])
one_hot = one_hot_encode(test_seq,8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


###Haciendo mini-batches para entrenamiento
Para entrenar esta data, vamos a crear mini-batches para el entrenamiento. Debemos tener en cuenta que nuestro batch debe tener múltiples secuencias de una cantidad de números.

###Creando batches
1. Primero vamos a tener que descartar algo de texto, para tener un batch completo.
2. Luego camos a dividir nuestro *arr* en N batches.
3. Ahora vamos a tener nuestro arreglo, podemos iterar por nuestros mini-batches.

In [10]:
def get_batches(arr, batch_size, seq_length):
    '''Creamos un generador que regresa el batches del tamaño
     (batch_size) x (seq_length del arr).
       
       Argumentos
       ---------
       arr: Arreglo que queremos crear batchs de el
       batch_size: Tamaño del Batch, el número de secuencia por batch
       seq_length: Número de caracteres en la secuencia
    '''
    
    batch_size_total = batch_size * seq_length
    # Número total de batch que podemos hacer
    n_batches = len(arr)//batch_size_total
    
    # Seleccionamos la cantidad de caracteres necesarios para llenar todo el batch
    arr = arr[:n_batches * batch_size_total]
    # Reshape en tamaño de columnas batch_size
    arr = arr.reshape((batch_size, -1))
    
    # Iteramos en el arreglo en una secuencia por vez
    for n in range(0, arr.shape[1], seq_length):
        # Las características
        x = arr[:, n:n+seq_length]
        # Los objetivos
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

<h4>Probemos nuestra implementación<h4>
Vamos a crear un tamaño batch de 8 y una secuencia de pasos de 50.

In [11]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# Mostramos las primeras 10 secuencias
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[50  4 58  4 30 48 40 30 15 42]
 [25 33 40 30 12 47 31 34 42 28]
 [30  1 47 42 30 53 28  4 53  4]
 [ 6 28 15 47 11 40 44 13 18  4]
 [11 34 25 40 25 42 15 30 25 42]
 [37 30 11 40 30 54 26 28 28 42]
 [40 58 34 11 34 40 30 48 30 25]
 [56 30 11 47 22 40 28 13 58 42]]

y
 [[ 4 58  4 30 48 40 30 15 42 30]
 [33 40 30 12 47 31 34 42 28  4]
 [ 1 47 42 30 53 28  4 53  4 56]
 [28 15 47 11 40 44 13 18  4 15]
 [34 25 40 25 42 15 30 25 42 30]
 [30 11 40 30 54 26 28 28 42 40]
 [58 34 11 34 40 30 48 30 25 42]
 [30 11 47 22 40 28 13 58 42 56]]


---
## Definamos la red con PyTorch

### Estructura del Modelo

En `__init__` usaremos la siguiente estructura:
* Crear y almacenar los diccionarios necesarios
* Definir la capa LSTM que coge los parametros: input_size (Número de caracteres), hidden layer `n_hidden`, número de capas `n_layers`, la propabilidad de dropout `dropout` y batch_first que será `True`
* Definamos la capa de dropout
* Definimos una red fully-connected con parametros: tamaño de entrada `n_hidden` y la salida (número de caracateres).
* Finalmente inicializamos los pesos

---
### LSTM Entradas/Salidas

Podemos crear una red [LSTM layer] (https://pytorch.org/docs/stable/nn.html#lstm) así

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

Donde `input_size` es el número de caracteres que se espera en la secuencia de entrada y `n_hidden` es el número de unidades de capas ocultas, podemos añadir droput. Finalmente en la función `forward` podemos apilar varias capas usando `.vieq`.
Además necesitamos crear un hidden state con ceros así:

```python
self.init_hidden()
```

In [13]:
# Verificamos GPU
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Entrenando en GPU!')
else: 
    print('No hay GPU disponible, entrenamiento en CPU; considere hacer n_epochs muy pequeños.')

Entrenando en GPU!


In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creando el diccionario de caracteres
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Definimos la LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Definimos la capa de dropout
        self.dropout = nn.Dropout(drop_prob)
        
        ## Definimos la capa densa
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Obtenemos el resultado y el nuevo hidden state
        r_output, hidden = self.lstm(x, hidden)
        
        ## pasamos por el droput
        out = self.dropout(r_output)
        
        # Apilamos las salidas del LSTM
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## Pasamos el resultado del LSTM por la FC
        out = self.fc(out)
        
        # retornamos los resultados
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Iniciamos hidden state '''
        # Creamos dos tensores con tamaño n_layers x batch_size x n_hidden,
        # E iniciamos con cero
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Tiempo de entrenar

La función de entrenamiento define el número de epochs, learning reate y otros parámetros.
Usaremos Adam optimizer y cross-entropy loss porque usamos una clasificación de caracteres.

Detalle del entrenamiento: 
>* Usaremos [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) para evitar la explosión del gradiente.

In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Entrenando una red
    
        Argumento
        ---------
        
        net: Red de caracteres RNN
        data: El texto que se va a entrenar la red
        epochs: Número de epochs que entrenará
        batch_size: Número de mini-batch
        seq_length: Número de caracteres por mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Dividir la data para validar
        print_every: Imprimir el loss y la validación
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # Creamos entrenamiento y validación
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # Iniciamos hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode y hacemos tensor
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creamos una nueva variable para hidden state
            h = tuple([each.data for each in h])

            # Enceramos el gradiente
            net.zero_grad()
            
            # Obtenemos las salidas
            output, h = net(inputs, h)
            
            # Calculamos la pérdida y optimizamos
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` preeve la explosión de gradiente, problema común en RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # La perdida LOSS
            if counter % print_every == 0:
                # loss validation 
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode y creamos Tensores
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creamos el hidden state,
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # Regresamos a entrenar
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instanciando el modelo

Asignnado los hiperparametros a nuesttro modelo

In [17]:
# Definamos e imprimimos la red
n_hidden=512
n_layers=2

net = CharRNN(caracteres, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(59, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=59, bias=True)
)


In [18]:
batch_size = 128
seq_length = 100
n_epochs = 30 # Aumentaremos al modelo final

# Entrenamos el modelo
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/30... Step: 10... Loss: 3.1550... Val Loss: 3.1056
Epoch: 1/30... Step: 20... Loss: 3.0803... Val Loss: 3.0514
Epoch: 1/30... Step: 30... Loss: 3.0530... Val Loss: 3.0365
Epoch: 1/30... Step: 40... Loss: 3.0402... Val Loss: 3.0313
Epoch: 2/30... Step: 50... Loss: 3.0405... Val Loss: 3.0288
Epoch: 2/30... Step: 60... Loss: 3.0356... Val Loss: 3.0260
Epoch: 2/30... Step: 70... Loss: 3.0283... Val Loss: 3.0204
Epoch: 2/30... Step: 80... Loss: 3.0176... Val Loss: 3.0019
Epoch: 2/30... Step: 90... Loss: 2.9790... Val Loss: 2.9544
Epoch: 3/30... Step: 100... Loss: 2.8776... Val Loss: 2.8542
Epoch: 3/30... Step: 110... Loss: 2.7410... Val Loss: 2.6933
Epoch: 3/30... Step: 120... Loss: 2.6954... Val Loss: 2.6504
Epoch: 3/30... Step: 130... Loss: 2.5883... Val Loss: 2.5383
Epoch: 3/30... Step: 140... Loss: 2.4930... Val Loss: 2.4511
Epoch: 4/30... Step: 150... Loss: 2.4127... Val Loss: 2.3703
Epoch: 4/30... Step: 160... Loss: 2.3422... Val Loss: 2.3328
Epoch: 4/30... Step: 170... Loss:

## Checkpoint

Después de entrenar debemos guardar nuestro modelo


In [19]:
# Ponemos un nombre y guardamos
nombre_modelo = 'rnn_lsmt.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(nombre_modelo, 'wb') as f:
    torch.save(checkpoint, f)

---
## Haciendo predicciones

¡Ahora que el modelo está entrenado, querremos probarlo y hacer predicciones sobre los próximos personajes! Para muestrear, pasamos un carácter y hacemos que la red prediga el siguiente carácter. Luego tomamos ese personaje, lo devolvemos y obtenemos otro personaje predicho. ¡Sigue haciendo esto y generarás un montón de texto!

### Nota sobre la función `predict`

La salida de nuestro RNN es de una capa completamente conectada y genera una **distribution of next-character scores**.

> Para obtener el siguiente carácter, aplicamos una función softmax, que nos da una distribución de *probabilidad* que luego podemos muestrear para predecir el siguiente carácter.

### Top K
Nuestro modelo usa una softmax, pero podemos añadir algo de aleatoridad para escoger uno de los caracteres más probables con Top K, asi puede generar un texto algo aleatorio y aveces absurdo. [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [20]:
def predict(net, char, h=None, top_k=None):
        ''' damos un caracter y predice el siguiente
            Regresa la predcción y el hidden state
        '''
        
        # Entrada de tensores
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # Quitar el hidden state del historial
        h = tuple([each.data for each in h])
        # Obtenemos la salida del modelo
        out, h = net(inputs, h)

        # Obtenemos la probabildad de caracteres
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # Regresa a la CPU
        
        # Obtenemos el top de caracteres
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # añadimos algo de aleatoridad al siguiente caracter
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # Devolvemos el hidden state y el caracter
        return net.int2char[char], h

### Generando texto

Al incio será más aleatorio, mientras más tiempo pase por la red más robusta será la predicción.


In [21]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # Evalua modelo
    
    # Los primeros caracteres
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Pasamos el caracter anterior y genera el siguiente
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [23]:
print(sample(net, 1000, prime='El libro empieza años', top_k=5))

El libro empieza años después de la fundación, en la época en que José Arcadio Buendía parecía haber superado ya su antigua obsesión por los
grandes inventos que traían a Macondo los gitanos de la tribu de Melquíades, artilugios sobradamente conocidos (como los imanes o el catalejo)
que no habían llegado todavía a aquella recóndita aldea. Deseoso de poner en contacto el pueblo con los avances de la civilización e ignorando
completamente la geografía de la región, José Arcadio Buendía había emprendido una fracasada expedición al Norte: encontraron únicamente tierras
inhóspitas y, a continuación, los restos de un galeón español y el mar; seguidamente, su proyecto de trasladar Macondo a algún lugar
menos aislado topó la férrea oposición de Úrsula.
Los primeros Buendía tuvieron tres hijos (José Arcadio, Aureliano y Amaranta), cuya infancia, adolescencia y
primera juventud se relata en esta primera parte. El mayor, llamado José Arcadio como su padre, nació durante
el viaje fundacional. Ya e

Como ya tenemos el modelo guardado en **rnn_lsmt.net** podemos hacer el uso de ello para generar textos.

In [25]:
# Cargamos el modelo
with open('rnn_lsmt.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [28]:
# Creamos nuevo texto con un texto inicial
print(sample(loaded, 1000, top_k=5, prime="El narrador se remonta"))

El narrador se remonta noca menos que al siglo XVI para presentarnos a los bisabuelos de los protagonistas,
pero la historia comienza Varios siglos más tarde, con la boda de los futuros fundadores de Macondo,
que eran primos entre sí: José Arcadio Buendía y Úrsula Iguarán.
Con nefastos augurios sobre su descendencia por su cercano parentesco
(unos tíos de los recién casados habían tenido un hijo con cola de cerdo),
la madre de Úrsula logró aterrorizar a su hija; enfundada en un inquebrantable cinturón de castidad,
y pese a las embestidas de José Arcadio, Úrsula impidió durante un año y medio la consumación del matrimonio.
Las habladurías atribuían la falta de hijos a la impotencia de José Arcadio, y un día Prudencio Aguilar,
después de perder una riña de gallos, se burló públicamente de su poca hombría. Tal ofensa da lugar
a un duelo de honor en que Prudencio Aguilar muere a manos de José Arcadio, quien obliga a continuación a su esposa a quitarse el cinturón.
Con la consumación del ma